In [11]:
#imports
import pandas as pd
from google.cloud import bigquery
from time import time
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/reneboygarcia/Library/CloudStorage/GoogleDrive-reneboygarcia@gmail.com/My Drive/Personal/Data Science Notebook/Data Engineering/week_3_data_warehouse/GOOGLE_APPLICATION_CREDENTIALS.json"
print("Setup Complete")

Setup Complete


In [4]:
#create a client
client = bigquery.Client()
# Construct a reference to the "ny_taxi_bucket_de_2023" dataset
dataset_ref = client.dataset(dataset_id="ny_taxi", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
#construct a table reference to
table_ref = dataset_ref.table('ny_taxi_tripdata_2019')
#API request
table_green_taxi_trips = client.get_table(table_ref)
#to dataframe
df_yellow_taxi_trips = client.list_rows(table=table_green_taxi_trips, max_results=5).to_dataframe()
df_yellow_taxi_trips.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-11-01 00:04:42+00:00,2019-11-01 00:13:53+00:00,N,1.0,95,196,1.0,1.51,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,1.0,0.0
1,2,2019-11-01 00:43:35+00:00,2019-11-01 00:43:39+00:00,N,1.0,193,7,1.0,0.00,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,1.0,0.0
2,1,2019-11-01 03:10:58+00:00,2019-11-01 03:13:59+00:00,N,5.0,129,129,1.0,0.70,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,2.0,0.0
3,1,2019-11-01 03:46:32+00:00,2019-11-01 03:54:20+00:00,N,5.0,129,173,1.0,1.00,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,2.0,0.0
4,1,2019-11-01 04:03:12+00:00,2019-11-01 04:07:50+00:00,N,5.0,129,129,1.0,0.60,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0,2.0,2.0,0.0


In [59]:
# query
query = """
        SELECT COUNT(1) as num_count
        FROM `dtc-de-2023.ny_taxi.ny_taxi.ny_taxi_tripdata_2019`
        WHERE DATE(tpep_pickup_datetime) BETWEEN "2019-01-01" AND "2019-01-01"
        LIMIT 100
        """
        
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
query_results = client.query(query, job_config=safe_config)
#convert to dataframe
df_query_results = query_results.to_dataframe()
df_query_results.head()

client.create_table()

,num_count
0,24827


In [22]:
# amount of data scanned in GB
def data_scanned(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    query_job_bytes = client.query(query, job_config=dry_run_config).total_bytes_processed
    print(f'Data processed: {round(query_job_bytes / 10**9, 3)} GB')

def runtime(query):
    time_config = bigquery.QueryJobConfig(use_query_cache=False)
    start = time()
    query_result = client.query(query, job_config=time_config).result()
    end = time()
    print(f'Runtime is: {round(end-start, 3)} seconds')

In [23]:
data_scanned(query=query)
runtime(query=query)

Data processed: 0.011 GB
Runtime is: 1.568 seconds


In [ ]:
# Construct a reference to the "prefect_de_2023" dataset
dataset_ref_gcs = bigquery.DatasetReference(project="dtc-de-2023", dataset_id="prefect-de-2023")
# construct a table reference to "green_tripdata_2019-04"
table_ref_gcs = dataset_ref_gcs.table("green_tripdata_2020-01")
# API request
green_ny_taxi_trips_table = bigquery.Table(table_ref_gcs)
external_config = bigquery.ExternalConfig("PARQUET")
external_config.source_uris = ["gs://prefect-de-2023/data/green/green_tripdata_2020-01.parquet"]
# green_ny_taxi_trips_table.external_data_configuration = external_config

# Create a permanent table linked to GCS file
table = client.create_table(table=green_ny_taxi_trips_table, )
# to DataFrame
df_green_ny_taxi_trips = client.list_rows(table=table, max_results=5).to_dataframe()
df_green_ny_taxi_trips.head()

# 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/dtc-de-2023/datasets/prefect-de-2023/tables?prettyPrint=false: Not found: Dataset dtc-de-2023:prefect-de-2023

In [ ]:
#create a client
client = bigquery.Client()
# Construct a reference to the "prefect_de_2023_datasetid" dataset
dataset_ref = client.dataset("prefect_de_2023_datasetid", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
# construct a table reference to "yellow_ny_taxi_trips_table"
table_ref = dataset_ref.table('yellow_ny_taxi_trips_table')
#API request
table_yellow_taxi_trips = client.get_table(table_ref)
#to dataframe
df_yellow_taxi_trips = client.list_rows(table=table_yellow_taxi_trips, max_results=5).to_dataframe()
df_yellow_taxi_trips.head()

In [57]:
# Downloads a blob to a file path.
from prefect import flow
from prefect_gcp import GcpCredentials
from prefect_gcp.cloud_storage import cloud_storage_download_blob_to_file, 
from pathlib import Path
@flow
def example_cloud_storage_download_blob_flow():
    gcp_credentials = GcpCredentials.load("ny-taxi-gcp-creds")  # inferred from env, or set service_account_file
    contents = cloud_storage_download_blob_to_file(bucket="ny_taxi_bucket_de_2023", blob="green/green_tripdata_2019-11.parquet", gcp_credentials=gcp_credentials, path=Path("./week3_hw"))
    return contents

example_cloud_storage_download_blob_flow()

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'example-cloud-storage-download-blob-flow' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_71701/1514260246.py:6' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


11:58:25.619 | INFO    | prefect.engine - Created flow run 'authentic-otter' for flow 'example-cloud-storage-download-blob-flow'

11:58:28.950 | INFO    | Flow run 'authentic-otter' - Created task run 'cloud_storage_download_blob_to_file-7cd7d622-0' for task 'cloud_storage_download_blob_to_file'

11:58:28.952 | INFO    | Flow run 'authentic-otter' - Executing 'cloud_storage_download_blob_to_file-7cd7d622-0' immediately...

11:58:30.221 | INFO    | Task run 'cloud_storage_download_blob_to_file-7cd7d622-0' - Downloading blob named green/green_tripdata_2019-11.parquet from the ny_taxi_bucket_de_2023 bucket to week3_hw

11:58:38.717 | INFO    | Task run 'cloud_storage_download_blob_to_file-7cd7d622-0' - Finished in state Completed()

11:58:39.077 | INFO    | Flow run 'authentic-otter' - Finished in state Completed()

PosixPath('week3_hw/green/green_tripdata_2019-11.parquet')

In [9]:
# read data from the web into Dataframe
from pathlib import Path
import urllib.request

dataset_url = (
        f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-04.parquet"
    )

def fetch(dataset_url):
    filename, _ = urllib.request.urlretrieve(dataset_url)
    return filename


In [123]:
fetch(dataset_url)

'/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/tmpk_dytf59'

In [14]:
path_name = fetch(dataset_url=dataset_url)

path_name.endswith(".parquet")

False

In [ ]:
def read_df(path: str, dataset_url: str) -> pd.DataFrame:
    if dataset_url.endswith(".parquet"):
        df = pd.read_parquet(path=path)
    elif dataset_url.endswith(".csv.gz"):
        df = pd.read_csv(path, engine="pyarrow", compression="gzip")
    else:
        df = pd.read_csv(path, engine="pyarrow")
    
    print(f"Number of rows: {df.shape[0]}")
    return df

path_name = fetch(dataset_url=dataset_url)
df_ = read_df(path_name)
df_

In [16]:
# 
from prefect import flow,task
from prefect_gcp import GcpCredentials
from prefect_gcp.cloud_storage import cloud_storage_upload_blob_from_file
from pathlib import Path
import urllib.request

@task
def fetch(dataset_url):
    filename, _ = urllib.request.urlretrieve(dataset_url)
    return filename

@flow()
def example_cloud_storage_upload_blob_flow():
    path = fetch("https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-01.csv.gz")
    gcp_credentials = GcpCredentials.load("ny-taxi-gcp-creds")  # inferred from env, or set service_account_file
    blob = cloud_storage_upload_blob_from_file(file=path, 
                                                bucket="ny_taxi_bucket_de_2023",
                                                blob="2019/fhv_tripdata_2019-01.parquet",
                                                gcp_credentials=gcp_credentials)
    return blob

example_cloud_storage_upload_blob_flow()


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/tasks.py:261: UserWarning: A task named 'fetch' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_33528/946018608.py:8' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'example-cloud-storage-upload-blob-flow' and defined at '/var/folders/h2/6110hgtx6b7400ybw7yjhn400000gn/T/ipykernel_33528/946018608.py:13' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


22:41:03.554 | INFO    | prefect.engine - Created flow run 'clay-tuatara' for flow 'example-cloud-storage-upload-blob-flow'

22:41:06.128 | INFO    | Flow run 'clay-tuatara' - Created task run 'fetch-b4598a4a-0' for task 'fetch'

22:41:06.132 | INFO    | Flow run 'clay-tuatara' - Executing 'fetch-b4598a4a-0' immediately...

22:42:49.786 | INFO    | Task run 'fetch-b4598a4a-0' - Finished in state Completed()

...

22:42:50.489 | INFO    | Flow run 'clay-tuatara' - Executing 'cloud_storage_upload_blob_from_file-9a1bf371-0' immediately...

22:42:51.541 | INFO    | Task run 'cloud_storage_upload_blob_from_file-9a1bf371-0' - Uploading blob named 2019/fhv_tripdata_2019-01.parquet to the ny_taxi_bucket_de_2023 bucket

22:44:38.912 | INFO    | Task run 'cloud_storage_upload_blob_from_file-9a1bf371-0' - Finished in state Completed()

22:44:39.278 | INFO    | Flow run 'clay-tuatara' - Finished in state Completed()

'2019/fhv_tripdata_2019-01.parquet'

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

table_id = "dtc-de-2023.ny_taxi.ny_taxi_tripdata_2019 "
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.PARQUET,
)
uri = "gs://ny_taxi_bucket_de_2023/2019/fhv_tripdata_2019-01.csv.gz"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)
print(f"Loaded {destination_table.num_rows} rows.")

In [ ]:
# USING Q4-OPTION2 - Query with PARTITION AND CLUSTER
# Query 
query_dedup = """ 
            -- CREATE A CTE TABLE
            CREATE OR REPLACE TABLE 
                `dtc-de-2023.dlb_lake.user_sessions` AS 
            WITH 
                CTE1 AS (
                SELECT 
                    *, 
                    ROW_NUMBER() OVER(
                                        PARTITION BY user_session_id 
                                        ORDER BY _bq_inserted_at) AS latest_row
                FROM `dtc-de-2023.dlb_lake.staging_user_sessions`)

            -- FETCH ONLY THE LATEST ROW WHICH IS THE LATEST BQ INSERTED TIMESTAMP
            SELECT * EXCEPT (latest_row)
            FROM CTE1
            WHERE latest_row = 1 
            """
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**12)
#query
results_query_dedup = client.query(query_dedup, job_config=safe_config)
